# Testing di un caso complesso: TetGrid

Si vuole testare l'efficacia della funzione spaceindex() su un modello geometrico complesso. Nello specifico si considera una griglia 2x2x2 dove sono costruiti due tetraedri randomici su ogni punto intero della stessa.

### Fase di creazione del modello

In [ ]:
using ViewerGL
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

La funzione t1t2(i,j,k) costruisce due tetraedri a partire dalle coordinate i,j,k di un punto intero

In [ ]:
function t1t2(i,j,k)
    v1 = rand(3,4).+ [2i,2j,2k]
    v2 = rand(3,4).+ [2i,2j,2k]
    ev = [[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
    fv = [[2,3,4],[1,3,4],[1,2,4],[1,2,3]]
    t1 = [v1,fv,ev] 
    t2 = [v2,fv,ev]
    return t1,t2
end

La funzione randomTetGrid() costruisce il modello e lo visualizza

In [ ]:
function randomTetGrid()

    N = 3  # N ≥ 2
    tets = []
    for i=0:N-1, j=0:N-1, k=0:N-1
        push!(tets, t1t2(i,j,k)...)
    end
    V,FV,EV = Lar.struct2lar(Lar.Struct(tets))
    
    #plot dei tetraedri
    GL.VIEW([ GL.GLGrid(V,FV, GL.COLORS[1],1), GL.GLFrame2 ]);

    W,(_,EW,FW,_) = Lar.cuboidGrid([N-1,N-1,N-1],true);
    U = (W .* 2) .+ 0.5
    
    #plot del modello
    GL.VIEW([ GL.GLGrid(V,FV,GL.COLORS[1],0.5), GL.GLGrid(U,FW,GL.COLORS[1],0.5), GL.GLFrame2 ]);

    V,FV,EV = Lar.struct2lar(Lar.Struct([Lar.Struct(tets), Lar.Struct([(U,FW,EW)]) ]))

    #plot degli spigoli modello 
    GL.VIEW([ GL.GLGrid(V,EV, GL.COLORS[1],0.5), GL.GLFrame2 ]);
    return (V,FV,EV)
end

In [ ]:
V,FV,EV = randomTetGrid()

### Fase di testing

![intersezione](images/bbintersect.png)

Per provare l’effettivo funzionamento di spaceindex bisogna controllare se il boundingbox i-esimo interseca tutti i boundingbox contenuti nella variabile covers[i], e se non interseca quelli che non sono contenuti in covers[i].
Di ogni boundingbox conosciamo una coppia di punti (N1, N2) che sono i punti opposti del parallelepipedo rispettivamente più vicini e più lontani dall’origine degli assi (figura 1).
Un boundingbox A interseca un boundingbox B se A contiene B (o viceversa), oppure se almeno uno dei suoi punti si trova all’interno di B (figura 2), cioè se, per ogni coordinata x, y e z, vale che 
N1b <= Na <= N2b

Inizialmente si esegue spaceindex() per ricavare le interzezioni tra i vari boundingbox

In [ ]:
using Test
include("../../src/intervaltree.jl")

cover = spaceindex((V,FV))
cells = [V[:,FV[k]] for k = 1:length(FV)]
bb = [hcat(boundingbox(c)...) for c in cells]

La funzione buildPoints(bb) si occupa di costruire tutte le coppie di vertici opposti del boundingbox bb, a partire dai due vertici conosciuti N1 e N2, attraverso delle opportune combinazioni delle coordinate.

In [ ]:
function buildPoints(bb)
    x1,y1,z1 = bb[1:3]
    x2,y2,z2 = bb[4:6]
    p1 = [x1 x2; y1 y2; z1 z2]
    p2 = [x2 x1; y1 y2; z1 z2]
    p3 = [x1 x2; y2 y1; z1 z2]
    p4 = [x1 x2; y1 y2; z2 z1]
    return p1,p2,p3,p4
end

hasIntersection(bb1, bb2) si occupa di controllare se bb1 e bb2 si intersecano utilizzando le proprietà sopra descritte. 

In [ ]:
function hasIntersection(bb1,bb2)
    #punti del boundingbox bb1
    x11,y11,z11 = bb1[1:3]
    x12,y12,z12 = bb1[4:6]
    #punti del boundingbox bb2
    x21,y21,z21 = bb2[1:3]
    x22,y22,z22 = bb2[4:6]

    #si controlla re il primo vertice di bb1 è contenuto in bb2
    #oppure se bb1 contiene bb2 
    first = ((x11>=x21 && x11<=x22) || (x11<=x21 && x12>=x22)) && ((y11>=y21 && y11<=y22) || (y11<=y21 && y12>=y22)) && ((z11>=z21 && z11<=z22) || (z11<=z21 && z12>=z22))
    #si controlla se il secondo vertice di bb1 è contenuto in bb2 
    second = (x12>=x21 && x12<=x22) && (y12>=y21 && y12<=y22) && (z12>=z21 && z12<=z22)

    return first || second
end

In [ ]:
#testing delle intersezioni
for j = 1:length(bb)
    for i = 1:length(cover[j])
        p1,p2,p3,p4 = buildPoints(bb[j])
        p5,p6,p7,p8 = buildPoints(bb[cover[j][i]])
        @test (hasIntersection(p1,bb[cover[j][i]]) || hasIntersection(p5,bb[j])
            || hasIntersection(p2,bb[cover[j][i]]) || hasIntersection(p6,bb[j]) 
            || hasIntersection(p3,bb[cover[j][i]]) || hasIntersection(p7,bb[j]) 
            || hasIntersection(p4,bb[cover[j][i]]) || hasIntersection(p8,bb[j]) ) == true
    end
end

In [ ]:
#testing delle non intersezioni
for j = 1:length(bb)
    noIntersection = setdiff((h for h = 1:length(FV)),cover[j])		#tutti i bb che non intersecano il j-esimo bb
    for i = 1:length(noIntersection)
        if bb[j] != bb[noIntersection[i]]
            p1,p2,p3,p4 = buildPoints(bb[j])
            p5,p6,p7,p8 = buildPoints(bb[noIntersection[i]])
            @test (hasIntersection(p1,bb[noIntersection[i]]) || hasIntersection(p5,bb[j])
                || hasIntersection(p2,bb[noIntersection[i]]) || hasIntersection(p6,bb[j]) 
                || hasIntersection(p3,bb[noIntersection[i]]) || hasIntersection(p7,bb[j]) 
                || hasIntersection(p4,bb[noIntersection[i]]) || hasIntersection(p8,bb[j]) ) == false
        end
    end
end